In [37]:
import numpy as np
import keras
from keras.api.datasets import mnist
from keras.api.utils import to_categorical

In [38]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [39]:
print("x_train shape", x_train.shape)
print("y_train shape", y_train.shape)
print("x_test shape", x_test.shape)
print("y_test shape", y_test.shape)

n_train_images = x_train.shape[0]
n_test_images = x_test.shape[0]

x_train shape (60000, 28, 28)
y_train shape (60000,)
x_test shape (10000, 28, 28)
y_test shape (10000,)


In [40]:
height = 28
width = 28
depth = 1

# Добавим третье измерение (глубину), изменим формат на float и нормализуем
x_train = x_train.reshape(n_train_images, height, width, depth).astype(np.float32) / 255
x_test = x_test.reshape(n_test_images, height, width, depth).astype(np.float32) / 255

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [41]:
n_classes = 10

print(y_train[0:5])

# Закодировать номер выходного класса массивом,
# где под индексом, равным номеру класса,
# стоит единица, а остальные элементы - нули
# (этот формат ещё называют one-hot)
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

print(y_train[0:5])

[5 0 4 1 9]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [ ]:
# Построение нейросети

model = keras.Sequential([
    keras.layers.Input(shape=(width, height, depth)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu"),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(n_classes, activation="softmax")
])

# Компиляция модели с заданеим функции потерь и оптимизации
# adam = optimizer=keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [61]:
batch_size = 128
n_epochs = 32

attempt = 3

save_path = f'checkpoints/CNN/attempt_{attempt}/ckpt_{batch_size}' + '_epoch_{epoch:02d}.keras'
save_callback = keras.callbacks.ModelCheckpoint(save_path, verbose=1, monitor='val_accuracy', mode='max', save_best_only=True)

# Обучение модели
model.fit(x_train,
          y_train,
          batch_size=batch_size,
          epochs=n_epochs,
          verbose=1,
          validation_split=0.1,
          callbacks = [save_callback])

Epoch 1/32
420/422 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7828 - loss: 0.6538
Epoch 1: val_accuracy improved from -inf to 0.98583, saving model to checkpoints/CNN/attempt_3/ckpt_128_epoch_01.keras
422/422 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.7837 - loss: 0.6513 - val_accuracy: 0.9858 - val_loss: 0.0510
Epoch 2/32
420/422 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9720 - loss: 0.0879
Epoch 2: val_accuracy improved from 0.98583 to 0.98867, saving model to checkpoints/CNN/attempt_3/ckpt_128_epoch_02.keras
422/422 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9721 - loss: 0.0878 - val_accuracy: 0.9887 - val_loss: 0.0408
Epoch 3/32
420/422 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9818 - loss: 0.0566
Epoch 3: val_accuracy improved from 0.98867 to 0.99267, saving model to checkpoints/CNN/attempt_3/ckpt_128_epoch_03.keras
422/422 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9818 - loss: 0.0566 - val_accuracy: 0.9927 - val_loss: 0.0302
Epoch 4/32
421/422 ━━━

In [62]:
# Оценка точности на тестовом датасете
model = keras.saving.load_model("checkpoints/CNN/attempt_3/ckpt_128_epoch_17.keras")
score = model.evaluate(x_test, y_test, verbose=0, return_dict=True)
print(score)

predicted_classes = model.predict(x_test)
correct_count = 0
print("Ошибки:")
for index, image in enumerate(predicted_classes):
    probability = max(image)
    actual_number = np.where(y_test[index] == 1)[0][0]
    prediction = np.where(image == probability)[0][0]
    if actual_number != prediction:
        print(f'Реальное число: {actual_number}, распознанное число: {prediction}, {probability * 100}%')
    else:
        correct_count += 1
print(f'\n{correct_count} из {n_test_images} распознано верно')

{'accuracy': 0.9954000115394592, 'loss': 0.013291829265654087}
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Ошибки:
Реальное число: 3, распознанное число: 5, 90.83303809165955%
Реальное число: 2, распознанное число: 4, 43.357595801353455%
Реальное число: 2, распознанное число: 1, 73.08472990989685%
Реальное число: 8, распознанное число: 9, 50.61575174331665%
Реальное число: 6, распознанное число: 5, 73.51993322372437%
Реальное число: 7, распознанное число: 1, 76.19256377220154%
Реальное число: 4, распознанное число: 6, 93.8510000705719%
Реальное число: 6, распознанное число: 5, 53.24708819389343%
Реальное число: 9, распознанное число: 4, 91.54093265533447%
Реальное число: 9, распознанное число: 5, 97.14086055755615%
Реальное число: 5, распознанное число: 7, 46.89332842826843%
Реальное число: 9, распознанное число: 5, 58.057695627212524%
Реальное число: 9, распознанное число: 4, 66.54836535453796%
Реальное число: 1, распознанное число: 2, 40.070635080337524%
Реальное число: 5, распознанное 